# Generador de archivos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import os
from datetime import datetime
import google.cloud.storage as storage
import json
import sys

# Ruta base en Google Drive
BASE_PATH = '/content/drive/MyDrive/Infra'

# Rutas para leer y escribir archivos
path = os.path.join(BASE_PATH, 'especificación.xlsx')
path_destino = os.path.join(BASE_PATH, 'generated_data/')

In [ ]:
# Cantidad de días desde la fecha actual hacia el pasado que generamos datos
cant_dias = 30

# Fecha actual
fecha_actual = datetime.now()

# Cantidad de distribuidores
cant_dist = 5

# Cantidad de clientes
cant_clientes = 10

# Cantidad sucursales
cant_sucursales = 25

In [ ]:
import csv
import numpy as np
import pandas as pd
import random as rd
import string as st
from datetime import datetime, timedelta
import names
from faker import Faker
import math

CIUDADES_PROVINCIAS = {
    'Buenos Aires': ['La Plata', 'Mar del Plata', 'Bahía Blanca', 'Tandil', 'Necochea', 'Olavarría', 'Junín', 'Tres Arroyos', 'Zárate', 'Campana'],
    'Santa Fe': ['Rosario', 'Santa Fe', 'Rafaela', 'Venado Tuerto', 'Reconquista', 'Santo Tomé', 'Villa Gobernador Gálvez', 'Sunchales', 'Esperanza', 'Villa Constitución'],
    'Córdoba': ['Córdoba', 'Río Cuarto', 'Villa María', 'San Francisco', 'Alta Gracia', 'Villa Carlos Paz', 'Bell Ville', 'Río Tercero', 'Villa Dolores', 'Jesús María'],
    'Mendoza': ['Mendoza', 'San Rafael', 'Godoy Cruz', 'Las Heras', 'Guaymallén', 'Luján de Cuyo', 'Maipú', 'Rivadavia', 'Tunuyán', 'San Martín'],
    'Tucumán': ['San Miguel de Tucumán', 'Yerba Buena', 'Concepción', 'Aguilares', 'Banda del Río Salí', 'Tafí Viejo', 'Monteros', 'Simoca', 'Famaillá', 'Lules'],
    'Salta': ['Salta', 'San Ramón de la Nueva Orán', 'Tartagal', 'Metán', 'Rosario de Lerma', 'Cafayate', 'Cerrillos', 'La Caldera', 'Campo Quijano', 'Aguaray'],
    'Jujuy': ['San Salvador de Jujuy', 'Palpalá', 'Perico', 'Libertador General San Martín', 'San Pedro de Jujuy', 'La Quiaca', 'Humahuaca', 'Tilcara', 'El Carmen', 'Fraile Pintado'],
    'Chaco': ['Resistencia', 'Barranqueras', 'Villa Ángela', 'Presidencia Roque Sáenz Peña', 'Charata', 'General San Martín', 'Quitilipi', 'Machagai', 'Las Breñas', 'La Leonesa'],
    'Corrientes': ['Corrientes', 'Goya', 'Mercedes', 'Curuzú Cuatiá', 'Paso de los Libres', 'Monte Caseros', 'Santo Tomé', 'Esquina', 'Empedrado', 'Bella Vista'],
    'Misiones': ['Posadas', 'Oberá', 'Eldorado', 'Puerto Iguazú', 'Apostoles', 'Leandro N. Alem', 'San Vicente', 'Jardín América', 'Aristóbulo del Valle', 'Puerto Rico'],
    'Entre Ríos': ['Paraná', 'Concordia', 'Gualeguaychú', 'Concepción del Uruguay', 'Gualeguay', 'Villaguay', 'Nogoyá', 'Federación', 'Victoria', 'Diamante'],
    'Formosa': ['Formosa', 'Clorinda', 'Pirané', 'El Colorado', 'Las Lomitas', 'Ingeniero Juárez', 'Laguna Blanca', 'Palo Santo', 'Villa General Güemes', 'Comandante Fontana'],
    'La Rioja': ['La Rioja', 'Chilecito', 'Aimogasta', 'Chamical', 'Chepes', 'Villa Unión', 'Famatina', 'Villa Sanagasta', 'Arauco', 'Vinchina'],
    'San Juan': ['San Juan', 'Rivadavia', 'Pocito', 'Rawson', 'Chimbas', 'Santa Lucía', 'Caucete', 'Albardón', '9 de Julio', 'Calingasta'],
    'San Luis': ['San Luis', 'Villa Mercedes', 'La Punta', 'Merlo', 'Concarán', 'Justo Daract', 'Tilisarao', 'Quines', 'Unión', 'Nueva Galia'],
    'Santiago del Estero': ['Santiago del Estero', 'La Banda', 'Frías', 'Añatuya', 'Termas de Río Hondo', 'Clodomira', 'Loreto', 'Quimilí', 'Sumampa', 'Villa Ojo de Agua'],
    'Catamarca': ['San Fernando del Valle de Catamarca', 'Valle Viejo', 'Recreo', 'Santa María', 'Andalgalá', 'Belgramo', 'Tinogasta', 'Fiambalá', 'Santa Rosa', 'Pomán'],
    'Neuquén': ['Neuquén', 'Cutral Có', 'Plottier', 'Centenario', 'Zapala', 'San Martín de los Andes', 'Villa La Angostura', 'Junín de los Andes', 'Aluminé', 'Chos Malal'],
    'Río Negro': ['Viedma', 'General Roca', 'Cipolletti', 'Bariloche', 'Allen', 'Villa Regina', 'Choele Choel', 'Cinco Saltos', 'Ingeniero Huergo', 'Lamarque'],
    'Chubut': ['Rawson', 'Comodoro Rivadavia', 'Trelew', 'Puerto Madryn', 'Esquel', 'Sarmiento', 'Dolavon', 'Gaiman', 'Trevelin', 'Lago Puelo'],
    'Santa Cruz': ['Río Gallegos', 'Caleta Olivia', 'Pico Truncado', 'Puerto Deseado', 'Las Heras', 'Perito Moreno', 'Puerto San Julián', 'El Calafate', 'Gobernador Gregores', 'Río Turbio'],
    'Tierra del Fuego': ['Ushuaia', 'Río Grande', 'Tolhuin']
}

# Leer las hojas de cálculo del archivo
df_sheets_desc = pd.read_excel(path, sheet_name=None)
df_cond_vta = df_sheets_desc['Condicion_Venta']
df_unidades = df_sheets_desc['Unidades']
df_tipo_negocio = df_sheets_desc['Tipo_Negocio']
df_cadena = df_sheets_desc['Cadena']
df_dias_visita = df_sheets_desc['Dias_Visita']
df_productos = df_sheets_desc['Productos']
df_cond_venta = df_sheets_desc['Condicion_Venta']
df_localidades = df_sheets_desc['Localidades']
df_estado = df_sheets_desc['Estado_cliente']
df_dias_visita = df_sheets_desc['Dias_Visita']

fake = Faker('es_AR')

MIN_STOCK_SAFETY = 50
MAX_STOCK_SAFETY = 200
MIN_DAILY_SALES = 0
MAX_DAILY_SALES = 150

SEASONAL_PATTERNS = {
    'SUMMER': {  # Diciembre, Enero, Febrero
        'beverages': 1.5,
        'snacks': 1.3,
        'cleaning': 1.2,
        'default': 1.1
    },
    'WINTER': {  # Junio, Julio, Agosto
        'beverages': 0.7,
        'snacks': 0.8,
        'cleaning': 1.3,
        'default': 0.9
    },
    'SPRING': {  # Septiembre, Octubre, Noviembre
        'beverages': 1.1,
        'snacks': 1.2,
        'cleaning': 1.1,
        'default': 1.0
    },
    'AUTUMN': {  # Marzo, Abril, Mayo
        'beverages': 0.9,
        'snacks': 0.9,
        'cleaning': 1.2,
        'default': 0.95
    }
}

PRODUCT_CATEGORIES = {
    'PREMIUM': {
        'price_range': (800, 2000),
        'stock_range': (100, 300),
        'daily_sales_range': (5, 30),
        'seasonality': 1.2,  # Mas afectado por las estaciones
        'stock_turnover': 0.8  # Lento rotación de stock
    },
    'STANDARD': {
        'price_range': (300, 799),
        'stock_range': (50, 200),
        'daily_sales_range': (10, 50),
        'seasonality': 1.0,
        'stock_turnover': 1.0
    },
    'BASIC': {
        'price_range': (100, 299),
        'stock_range': (30, 150),
        'daily_sales_range': (15, 70),
        'seasonality': 0.8,  # Menos afectado por las estaciones
        'stock_turnover': 1.2  # Rápido rotación de stock
    }
}

BUSINESS_TYPES = {
    'Supermercado': {
        'stock_multiplier': 2.0,
        'sales_multiplier': 2.5,
        'debt_probability': 0.1,
        'seasonality': 1.1,  # Mas afectado por las estaciones
        'stock_turnover': 1.2  # Rápido rotación de stock
    },
    'Mayorista': {
        'stock_multiplier': 3.0,
        'sales_multiplier': 3.0,
        'debt_probability': 0.15,
        'seasonality': 0.9,  # Menos afectado por las estaciones
        'stock_turnover': 1.1
    },
    'Distribuidora': {
        'stock_multiplier': 1.5,
        'sales_multiplier': 1.8,
        'debt_probability': 0.2,
        'seasonality': 1.0,
        'stock_turnover': 1.0
    },
    'Comercial': {
        'stock_multiplier': 1.0,
        'sales_multiplier': 1.0,
        'debt_probability': 0.25,
        'seasonality': 1.1,
        'stock_turnover': 0.9
    },
    'Almacén': {
        'stock_multiplier': 0.8,
        'sales_multiplier': 0.8,
        'debt_probability': 0.3,
        'seasonality': 1.2,
        'stock_turnover': 0.8
    }
}

PRODUCT_RELATIONSHIPS = {
    'complementary': {
        'beverages': ['snacks', 'cleaning'],
        'snacks': ['beverages', 'cleaning'],
        'cleaning': ['beverages', 'snacks']
    },
    'substitute': {
        'beverages': ['beverages'],
        'snacks': ['snacks'],
        'cleaning': ['cleaning']
    }
}

def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'SUMMER'
    elif month in [3, 4, 5]:
        return 'AUTUMN'
    elif month in [6, 7, 8]:
        return 'WINTER'
    else:
        return 'SPRING'

def get_product_type(producto):
    producto_lower = producto.lower()
    if any(word in producto_lower for word in ['bebida', 'gaseosa', 'jugo', 'cerveza']):
        return 'beverages'
    elif any(word in producto_lower for word in ['snack', 'galleta', 'chocolate', 'dulce']):
        return 'snacks'
    elif any(word in producto_lower for word in ['limpieza', 'detergente', 'jabón']):
        return 'cleaning'
    return 'default'

def calcular_factor_estacional(fecha, tipo_producto):
    season = get_season(fecha)
    return SEASONAL_PATTERNS[season].get(tipo_producto, SEASONAL_PATTERNS[season]['default'])

def calcular_factor_negocio(tipo_negocio, fecha):
    business_type = BUSINESS_TYPES.get(tipo_negocio, {})
    seasonality = business_type.get('seasonality', 1.0)
    return seasonality

def calcular_factor_producto(categoria_producto):
    return PRODUCT_CATEGORIES[categoria_producto].get('seasonality', 1.0)

def calcular_factor_rotacion(tipo_negocio, categoria_producto):
    business_turnover = BUSINESS_TYPES.get(tipo_negocio, {}).get('stock_turnover', 1.0)
    product_turnover = PRODUCT_CATEGORIES[categoria_producto].get('stock_turnover', 1.0)
    return business_turnover * product_turnover

def generar_codigos_sku(productos):
    codigos_sku = {}
    for producto in productos:
        # Generar SKUs mas estructurados (ej., CAT-XXXXX-YY)
        categoria = ''.join(rd.choice(st.ascii_uppercase) for _ in range(3))
        numero = ''.join(rd.choice(st.digits) for _ in range(5))
        version = ''.join(rd.choice(st.digits) for _ in range(2))
        sku_codigo = f"{categoria}-{numero}-{version}"
        codigos_sku[producto] = sku_codigo
    return codigos_sku

def generar_nombre_empresa(tipo_negocio):
    # Generar nombres de empresas más variados y específicos según el tipo de negocio
    if tipo_negocio == 'Supermercado':
        prefixes = ['Super', 'Mega', 'Maxi', 'Express']
        names = ['Market', 'Shopping', 'Center', 'Store']
    elif tipo_negocio == 'Mayorista':
        prefixes = ['Distribuidora', 'Mayorista', 'Comercial']
        names = ['Norte', 'Sur', 'Este', 'Oeste', 'Central']
    else:
        prefixes = ['Comercial', 'Distribuidora', 'Almacén']
        names = ['Norte', 'Sur', 'Este', 'Oeste', 'Central']

    prefix = rd.choice(prefixes)
    name = rd.choice(names)
    return f"{prefix} {name}"

def generar_cuit():
    # Generar un CUIT argentino con un formato más realista
    prefijo = rd.choice(['20', '23', '24', '25', '26', '27', '30', '33', '34'])
    numero = ''.join(rd.choice(st.digits) for _ in range(8))
    verificador = rd.choice(st.digits)
    return f"{prefijo}{numero}{verificador}"

def calcular_precio_venta(unidades, precio_base, tipo_negocio):
    # Agregar descuentos basados en la cantidad de unidades y tipo de negocio
    if unidades > 50:
        descuento = rd.uniform(0.05, 0.15)  # 5-15% discount for bulk
    elif unidades > 20:
        descuento = rd.uniform(0.02, 0.08)  # 2-8% discount for medium quantity
    else:
        descuento = 0

    business_multiplier = BUSINESS_TYPES.get(tipo_negocio, {}).get('sales_multiplier', 1.0)
    return round(precio_base * unidades * (1 - descuento) * business_multiplier, 2)

def generar_coordenadas_argentina(provincia):
    # Definir coordenadas aproximadas para las provincias argentinas
    COORDINATES = {
        'Buenos Aires': {'lat': (-38, -34), 'lon': (-62, -57)},
        'Córdoba': {'lat': (-33, -30), 'lon': (-65, -62)},
        'Santa Fe': {'lat': (-34, -30), 'lon': (-62, -60)},
        'Mendoza': {'lat': (-35, -32), 'lon': (-70, -68)},
        'Tucumán': {'lat': (-27, -26), 'lon': (-66, -65)},
        'Salta': {'lat': (-25, -22), 'lon': (-66, -63)},
        'Jujuy': {'lat': (-24, -22), 'lon': (-66, -65)},
        'Chaco': {'lat': (-27, -25), 'lon': (-61, -58)},
        'Corrientes': {'lat': (-30, -27), 'lon': (-59, -56)},
        'Misiones': {'lat': (-27, -25), 'lon': (-56, -54)},
        'Entre Ríos': {'lat': (-33, -30), 'lon': (-60, -58)},
        'Formosa': {'lat': (-26, -25), 'lon': (-58, -57)},
        'La Rioja': {'lat': (-31, -29), 'lon': (-67, -66)},
        'San Juan': {'lat': (-32, -30), 'lon': (-69, -68)},
        'San Luis': {'lat': (-34, -33), 'lon': (-67, -66)},
        'Santiago del Estero': {'lat': (-28, -27), 'lon': (-64, -63)},
        'Catamarca': {'lat': (-29, -27), 'lon': (-67, -66)},
        'Neuquén': {'lat': (-40, -38), 'lon': (-71, -69)},
        'Río Negro': {'lat': (-41, -38), 'lon': (-71, -67)},
        'Chubut': {'lat': (-45, -42), 'lon': (-71, -67)},
        'Santa Cruz': {'lat': (-52, -46), 'lon': (-72, -67)},
        'Tierra del Fuego': {'lat': (-54, -53), 'lon': (-68, -67)}
    }

    if provincia in COORDINATES:
        coords = COORDINATES[provincia]
        lat = rd.uniform(coords['lat'][0], coords['lat'][1])
        lon = rd.uniform(coords['lon'][0], coords['lon'][1])
    else:
        lat = rd.uniform(-55, -22)
        lon = rd.uniform(-73, -53)

    return round(lat, 6), round(lon, 6)

def generar_stock_inicial(unidad, tipo_negocio, categoria_producto, fecha, producto):
    base_range = PRODUCT_CATEGORIES[categoria_producto]['stock_range']
    multiplier = BUSINESS_TYPES.get(tipo_negocio, {}).get('stock_multiplier', 1.0)

    tipo_producto = get_product_type(producto)
    seasonal_factor = calcular_factor_estacional(fecha, tipo_producto)
    business_factor = calcular_factor_negocio(tipo_negocio, fecha)
    rotation_factor = calcular_factor_rotacion(tipo_negocio, categoria_producto)

    adjusted_min = int(base_range[0] * multiplier * seasonal_factor * business_factor)
    adjusted_max = int(base_range[1] * multiplier * seasonal_factor * business_factor)

    if unidad == "UNI":
        return rd.randint(adjusted_min, adjusted_max)
    else: 
        return rd.randint(int(10 * multiplier), int(50 * multiplier))

def generar_ventas_realistas(stock_inicial, unidad, tipo_negocio, categoria_producto, fecha, producto):
    base_range = PRODUCT_CATEGORIES[categoria_producto]['daily_sales_range']
    multiplier = BUSINESS_TYPES.get(tipo_negocio, {}).get('sales_multiplier', 1.0)

    tipo_producto = get_product_type(producto)
    seasonal_factor = calcular_factor_estacional(fecha, tipo_producto)
    business_factor = calcular_factor_negocio(tipo_negocio, fecha)
    rotation_factor = calcular_factor_rotacion(tipo_negocio, categoria_producto)

    adjusted_min = int(base_range[0] * multiplier * seasonal_factor * business_factor)
    adjusted_max = int(base_range[1] * multiplier * seasonal_factor * business_factor)

    if unidad == "UNI":
        max_venta = min(stock_inicial, adjusted_max)
        return rd.randint(adjusted_min, max_venta)
    else:
        max_venta = min(stock_inicial, int(30 * multiplier))
        return rd.randint(0, max_venta) 

def generar_telefono_argentino(provincia):
    # Definir códigos de área por provincia
    AREA_CODES = {
        'Buenos Aires': ['11', '15', '16', '17', '18', '19'],
        'Córdoba': ['35', '36', '37'],
        'Santa Fe': ['34'],
        'Mendoza': ['26'],
        'Tucumán': ['38'],
        'Salta': ['38'],
        'Jujuy': ['38'],
        'Chaco': ['36'],
        'Corrientes': ['37'],
        'Misiones': ['37'],
        'Entre Ríos': ['34'],
        'Formosa': ['37'],
        'La Rioja': ['38'],
        'San Juan': ['26'],
        'San Luis': ['26'],
        'Santiago del Estero': ['38'],
        'Catamarca': ['38'],
        'Neuquén': ['29'],
        'Río Negro': ['29'],
        'Chubut': ['29'],
        'Santa Cruz': ['29'],
        'Tierra del Fuego': ['29']
    }

    area_codes = AREA_CODES.get(provincia, ['11'])
    area = rd.choice(area_codes)
    number = ''.join(rd.choice(st.digits) for _ in range(7))
    return f"{area}-{number}"

def determinar_categoria_producto(producto):
    # Mapear productos con categorias en base de sus nombres o categorias
    premium_keywords = ['premium', 'exclusive', 'top', 'elite']
    basic_keywords = ['basic', 'simple', 'standard']

    producto_lower = producto.lower()
    if any(keyword in producto_lower for keyword in premium_keywords):
        return 'PREMIUM'
    elif any(keyword in producto_lower for keyword in basic_keywords):
        return 'BASIC'
    else:
        return 'STANDARD'

codigos_sku = generar_codigos_sku(df_productos['nombre'])

columns_v = df_sheets_desc['Desc_VentaClientes']['Campo']
columns_s = df_sheets_desc['Desc_StockClientes']['Campo']
columns_m = df_sheets_desc['Desc_Maestro']['Campo']


for distribuidor in range(1, cant_dist + 1):
    clientes_data = []
    for _ in range(cant_clientes):
        lat, lon = generar_coordenadas_argentina(rd.choice(list(CIUDADES_PROVINCIAS.keys())))
        cliente_data = {
            'nombre': names.get_full_name(),
            'empresa': generar_nombre_empresa(rd.choice(df_tipo_negocio['nombre'])),
            'cuit': generar_cuit(),
            'direccion': fake.street_address(),
            'telefono': generar_telefono_argentino(rd.choice(list(CIUDADES_PROVINCIAS.keys()))),
            'lat': lat,
            'lon': lon,
            'tipo_negocio': rd.choice(df_tipo_negocio['nombre']),
            'fecha_alta': fake.date_between(start_date='-2y', end_date='today')
        }
        clientes_data.append(cliente_data)

    for d in range(0,cant_dias):
        fech_cierre = fecha_actual - timedelta(days=d)
        fech_cierre_string = f'{fech_cierre.year}-{fech_cierre.month:02d}-{fech_cierre.day:02d}'
        fech_cie_com_data = [fech_cierre_string for _ in range(cant_clientes)]

        sucursales_dist = [(rd.randint((cant_sucursales*distribuidor)+1,cant_sucursales*(distribuidor+1))) for _ in range(cant_clientes)]

        clientes = [(rd.randint(1000,9999)) for _ in range(cant_clientes)]

        provincia = [(rd.choice(list(CIUDADES_PROVINCIAS.keys()))) for _ in range(cant_clientes)]
        ciudades_aleatorias = []
        for prov in provincia:
            ciudades_provincia = CIUDADES_PROVINCIAS[prov]
            ciudad_aleatoria = rd.choice(ciudades_provincia)
            ciudades_aleatorias.append(ciudad_aleatoria)

        estado = [(rd.choices(df_estado['nombre'], weights=df_estado['probabilidades'], k=1)[0]) for _ in range(cant_clientes)]

        unidad = [(rd.choice(df_unidades['codigo_unidad'])) for _ in range(cant_clientes)]

        stock = []
        vta_unidades_data = []
        vta_importe_data = []

        for i in range(cant_clientes):
            tipo_negocio = clientes_data[i]['tipo_negocio']
            categoria_producto = determinar_categoria_producto(df_productos['nombre'][i])
            producto = df_productos['nombre'][i]

            stock_inicial = generar_stock_inicial(unidad[i], tipo_negocio, categoria_producto, fech_cierre, producto)
            ventas = generar_ventas_realistas(stock_inicial, unidad[i], tipo_negocio, categoria_producto, fech_cierre, producto)
            precio_base = rd.uniform(
                PRODUCT_CATEGORIES[categoria_producto]['price_range'][0],
                PRODUCT_CATEGORIES[categoria_producto]['price_range'][1]
            )
            vta_importe = calcular_precio_venta(ventas, precio_base, tipo_negocio)

            stock_depletion = int(stock_inicial * (1 - (d / cant_dias))) 
            stock.append(max(0, stock_depletion - ventas))
            vta_unidades_data.append(ventas)
            vta_importe_data.append(vta_importe)

        cond_vta_data = [rd.choice(df_cond_vta['codigo_condicion']) for _ in range(cant_clientes)]

        dias_visita = [(rd.choice(df_dias_visita['codigo_dia'])) for _ in range(cant_clientes)]

        deuda_vencida = []
        for i in range(cant_clientes):
            tipo_negocio = clientes_data[i]['tipo_negocio']
            debt_prob = BUSINESS_TYPES.get(tipo_negocio, {}).get('debt_probability', 0.2)
            # Aumentar la probabilidad base de deuda
            debt_prob = min(0.7, debt_prob * 2)  # Aumentar la probabilidad pero no más del 70%
            if rd.random() < debt_prob:
                deuda = round(rd.uniform(1000, 10000), 2)
            else:
                deuda = 0
            deuda_vencida.append(deuda)

        producto = []
        sku_codigo = []
        producto1 = list(codigos_sku.keys())
        sku_codigo1 = list(codigos_sku.values())
        producto.extend(producto1)
        sku_codigo.extend(sku_codigo1)

        venta_clientes_data = list(zip(sucursales_dist,clientes,fech_cie_com_data,sku_codigo,vta_unidades_data,vta_importe_data,cond_vta_data))
        stock_data = list(zip(sucursales_dist, fech_cie_com_data, sku_codigo, producto, stock, unidad))
        maestro_data = list(zip(
            sucursales_dist,
            clientes,
            ciudades_aleatorias,
            provincia,
            estado,
            [c['nombre'] for c in clientes_data],
            [c['cuit'] for c in clientes_data],
            [c['empresa'] for c in clientes_data],
            [c['direccion'] for c in clientes_data],
            dias_visita,
            [c['telefono'] for c in clientes_data],
            [c['fecha_alta'].strftime('%Y-%m-%d') for c in clientes_data],
            ['ver' for _ in range(cant_clientes)],
            [c['lat'] for c in clientes_data],
            [c['lon'] for c in clientes_data],
            cond_vta_data,
            deuda_vencida,
            [c['tipo_negocio'] for c in clientes_data]
        ))

        df_vta_cli = pd.DataFrame(venta_clientes_data,columns=columns_v)
        df_stock = pd.DataFrame(stock_data,columns=columns_s)
        df_maestro = pd.DataFrame(maestro_data,columns=columns_m)

        carpeta_destino_v = os.path.join(path_destino, 'Archivos_VentaClientes', f'Distribuidor_{distribuidor}')
        carpeta_destino_s = os.path.join(path_destino, 'Archivos_Stock', f'Distribuidor_{distribuidor}')
        carpeta_destino_m = os.path.join(path_destino, 'Archivos_Maestro', f'Distribuidor_{distribuidor}')
        archivo_destino_v = os.path.join(carpeta_destino_v, f'Venta_Clientes_{fech_cierre_string}.csv')
        archivo_destino_s = os.path.join(carpeta_destino_s, f'StockPeriodo_{fech_cierre_string}.csv')
        archivo_destino_m = os.path.join(carpeta_destino_m, f'Maestro_{fech_cierre_string}.csv')

        try:
            os.makedirs(carpeta_destino_v, exist_ok=True)
            os.makedirs(carpeta_destino_s, exist_ok=True)
            os.makedirs(carpeta_destino_m, exist_ok=True)
        except Exception as e:
            print(f"Error creating directories: {e}")
            continue

        try:
            df_vta_cli.to_csv(archivo_destino_v, encoding='utf-8', index=False)
            df_stock.to_csv(archivo_destino_s, encoding='utf-8', index=False)
            df_maestro.to_csv(archivo_destino_m, encoding='utf-8', index=False)
            print(f"Generated files for Distribuidor_{distribuidor} - {fech_cierre_string}")
        except Exception as e:
            print(f"Error saving files: {e}")
            continue

print("Data generation completed!")

Generated files for Distribuidor_1 - 2025-06-12
Generated files for Distribuidor_1 - 2025-06-11
Generated files for Distribuidor_1 - 2025-06-10
Generated files for Distribuidor_1 - 2025-06-09
Generated files for Distribuidor_1 - 2025-06-08
Generated files for Distribuidor_1 - 2025-06-07
Generated files for Distribuidor_1 - 2025-06-06
Generated files for Distribuidor_1 - 2025-06-05
Generated files for Distribuidor_1 - 2025-06-04
Generated files for Distribuidor_1 - 2025-06-03
Generated files for Distribuidor_1 - 2025-06-02
Generated files for Distribuidor_1 - 2025-06-01
Generated files for Distribuidor_1 - 2025-05-31
Generated files for Distribuidor_1 - 2025-05-30
Generated files for Distribuidor_1 - 2025-05-29
Generated files for Distribuidor_1 - 2025-05-28
Generated files for Distribuidor_1 - 2025-05-27
Generated files for Distribuidor_1 - 2025-05-26
Generated files for Distribuidor_1 - 2025-05-25
Generated files for Distribuidor_1 - 2025-05-24
Generated files for Distribuidor_1 - 202

In [ ]:
df_stock.head()

In [ ]:
PATH = os.path.join(os.getcwd(),'/usm-infra-grupo2-275f9c8b5ef9.json')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = PATH

In [ ]:
storage_client = storage.Client()
bucket = storage_client.create_bucket('bucket_grupo2_infra')

In [ ]:
project_id = 'bucket_grupo2_infra'
client = storage.Client(project=project_id)

In [ ]:
from google.cloud import storage
import os

def upload_generated_data_to_bucket(bucket_name, generated_data_path):
    client = storage.Client()
    bucket = client.get_bucket('bucket_grupo2_infra')

    for root, dirs, files in os.walk(generated_data_path):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, generated_data_path)
            blob_path = f"generated_data/{relative_path}"
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_path)
            print(f"Subido: {blob_path}")

generated_data_path = "/content/drive/MyDrive/Infra/generated_data"
bucket_name = 'bucket_grupo2_infra'
upload_generated_data_to_bucket(bucket_name, generated_data_path)

Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-05-29.csv
Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-05-26.csv
Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-05-28.csv
Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-05-27.csv
Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-05-25.csv
Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-06-12.csv
Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-06-11.csv
Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-06-10.csv
Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-06-09.csv
Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-06-08.csv
Subido: generated_data/Archivos_VentaClientes/Distribuidor_1/Venta_Clientes_2025-06-07.csv